In [30]:
import torch
import torch.nn as nn

from torch.autograd import Function
from torchvision import datasets, transforms

In [26]:
class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

In [72]:
class DANN(nn.Module):
    def __init__(self):
        super(DANN, self).__init__()
        self.feature = nn.Sequential(
                    nn.Conv2d(3, 64, kernel_size=5),
                    nn.BatchNorm2d(64),
                    nn.MaxPool2d(2),
                    nn.ReLU(True),
                    nn.Conv2d(64, 50, kernel_size=5),
                    nn.BatchNorm2d(50),
                    nn.Dropout2d(),
                    nn.MaxPool2d(2),
                    nn.ReLU(True)
                )
        self.classifier = nn.Sequential(
                    nn.Linear(50 * 4 * 4, 100),
                    nn.BatchNorm1d(100),
                    nn.ReLU(True),
                    nn.Dropout(),
                    nn.Linear(100, 100),
                    nn.BatchNorm1d(100),
                    nn.ReLU(True),
                    nn.Linear(100, 10),
                    nn.LogSoftmax(dim=1)
                )
        self.domain_classifier = nn.Sequential(
                    nn.Linear(50 * 4 * 4, 100),
                    nn.BatchNorm1d(100),
                    nn.ReLU(True),
                    nn.Linear(100, 2),
                    nn.LogSoftmax(dim=1)
                )
        
    def forward(self, input_data, alpha):
            input_data = input_data.expand(input_data.data.shape[0], 3, 28, 28)
            feature = self.feature(input_data)
            feature = feature.view(-1, 50 * 4 * 4)
            reverse_feature = ReverseLayerF.apply(feature, alpha)
            class_output = self.classifier(feature)
            domain_output = self.domain_classifier(reverse_feature)

            return class_output, domain_output
        

In [29]:
dann = DANN()

In [46]:
import load_mnist_data
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import numpy as np
import torch.backends.cudnn as cudnn
import random
import os
import sys

In [33]:
x_train, x_test, mnist_m_x_train, mnist_m_x_test, y_train, y_test = load_mnist_data.get_numpy_data()

In [61]:
x_test.shape

(10000, 28, 28, 3)

In [66]:
x_train = x_train.reshape((60000, 3, 28, 28))
x_test = x_test.reshape((10000, 3, 28, 28))
mnist_m_x_train = mnist_m_x_train.reshape((60000, 3, 28, 28))
mnist_m_x_test = mnist_m_x_test.reshape((10000, 3, 28, 28))

In [67]:
mnist_X_train = torch.Tensor(x_train)
mnist_X_test =  torch.Tensor(x_test)
mnist_m_X_train = torch.Tensor(mnist_m_x_train)
mnist_m_X_test = torch.Tensor(mnist_m_x_test)
train_label = torch.Tensor(y_train)
test_label = torch.Tensor(y_test)

In [42]:
img_transform_source = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))
])

img_transform_target = transforms.Compose([
    transforms.Resize(28),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

In [68]:
kwargs = {}
if (torch.cuda.is_available()):
    kwargs = {'num_workers': 2, 'pin_memory': True}

mnist_dataset = TensorDataset(mnist_X_train,train_label)
dataloader_source = DataLoader(mnist_dataset, batch_size=128, **kwargs)

mnist_m_dataset = TensorDataset(mnist_m_X_train,train_label)
dataloader_target = DataLoader(mnist_m_dataset, batch_size=128, **kwargs)

In [86]:
lr = 1e-3
cuda = False
n_epoch = 100
cudnn.benchmark = True

my_net = DANN()
optimizer = optim.Adam(my_net.parameters(), lr=lr)

loss_class = torch.nn.NLLLoss()
loss_domain = torch.nn.NLLLoss()

if cuda:
    my_net = my_net.cuda()
    loss_class = loss_class.cuda()
    loss_domain = loss_domain.cuda()

for p in my_net.parameters():
    p.requires_grad = True


loss_class = torch.nn.NLLLoss()
loss_domain = torch.nn.NLLLoss()

if cuda:
    my_net = my_net.cuda()
    loss_class = loss_class.cuda()
    loss_domain = loss_domain.cuda()

for p in my_net.parameters():
    p.requires_grad = True

In [90]:
# training
best_accu_t = 0.0
for epoch in range(n_epoch):

    len_dataloader = min(len(dataloader_source), len(dataloader_target))
    data_source_iter = iter(dataloader_source)
    data_target_iter = iter(dataloader_target)

    for i in range(len_dataloader):

        p = float(i + epoch * len_dataloader) / n_epoch / len_dataloader
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        # training model using source data
        data_source = data_source_iter.next()
        s_img, s_label = data_source

        my_net.zero_grad()
        batch_size = len(s_label)

        domain_label = torch.zeros(batch_size).long()

        if cuda:
            s_img = s_img.cuda()
            s_label = s_label.cuda()
            domain_label = domain_label.cuda()

        class_output, domain_output = my_net.forward(input_data=s_img, alpha=alpha)

        class_output = class_output.type(torch.LongTensor)
        s_label = s_label.type(torch.LongTensor)

        print(class_output[0])
        print(s_label[0])

        err_s_label = loss_class(class_output, s_label)
        err_s_domain = loss_domain(domain_output, domain_label)

        # training model using target data
        data_target = data_target_iter.next()
        t_img, _ = data_target

        batch_size = len(t_img)

        domain_label = torch.ones(batch_size).long()

        if cuda:
            t_img = t_img.cuda()
            domain_label = domain_label.cuda()

        _, domain_output = my_net.forward(input_data=t_img, alpha=alpha)
        domain_output.type(torch.LongTensor)

        err_t_domain = loss_domain(domain_output, domain_label)
        err = err_t_domain + err_s_domain + err_s_label
        err.backward()
        optimizer.step()

        sys.stdout.write('\r epoch: %d, [iter: %d / all %d], err_s_label: %f, err_s_domain: %f, err_t_domain: %f' \
              % (epoch, i + 1, len_dataloader, err_s_label.data.cpu().numpy(),
                 err_s_domain.data.cpu().numpy(), err_t_domain.data.cpu().item()))
        sys.stdout.flush()

tensor([-2.1139, -1.9446, -2.6022, -2.1584, -2.5155, -2.0871, -2.6167, -2.5285,
        -2.2533, -2.4789], grad_fn=<SelectBackward0>)
tensor(5.)


RuntimeError: expected scalar type Long but found Float

In [ ]:
def train(trainloader, net, criterion, optimizer, use_cuda=True):
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        batch_size = inputs.size(0)
        total += batch_size
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*batch_size
        _, predicted = torch.max(outputs.data, 1)
        correct += predicted.eq(targets.data).cpu().sum().item()

    return train_loss/total, 100 - 100.*correct/total